### 2.3.2 Tensor



In [1]:
import torch

#  x 是叶子结点
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [2]:
# y经过运算
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [3]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [4]:
# 通过tensor.requires_grad_(True) 来增加grad_fn属性
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)

False
True


In [6]:
print(out)
out.backward()
print(x.grad)

tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [7]:
# grad 是累加的
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


In [12]:
# 一个常用的实际例子
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

# z不是一维的，所以需要传入同形的权重向量
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)   # z反向传播
print(x.grad)   # 输出相对于x的梯度

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)
tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [13]:
# 停止梯度追踪的例子
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

# 将y3梯度回传
y3.backward()
print(x.grad)

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [15]:
# 若我们想修改tensor的值，但是又不希望被autograd记录（即不影响反向传播），则只需对tensor.data进行操作
x = torch.ones(1, requires_grad=True)

print(x.data)
print(x.data.requires_grad)

y = 2 * x
x.data *= 100 # 只改变了值，并没有改变梯度

y.backward()
print(x)    # 更改data值也会影响tensor值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
